## 1. Loading and Exploring the Data

In [1]:
import pandas as pd

# Loading dataset
file_path = 'olist_order_reviews_dataset.csv'
df = pd.read_csv(file_path)

# Visualizing the first lines and information
print(df.head())
print(df.info())

# Checking null values
print(df.isnull().sum())

                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   

   review_score review_comment_title  \
0             4                  NaN   
1             5                  NaN   
2             5                  NaN   
3             5                  NaN   
4             5                  NaN   

                              review_comment_message review_creation_date  \
0                                                NaN  2018-01-18 00:00:00   
1                                                NaN  2018-03-10 00:00:00   
2                                                NaN  2018-02-17 00:00:00   
3           

## 2. Pre-processing and Creating Labels

In [2]:
# Removing lines where the comment is null
df_reviews = df.dropna(subset=['review_comment_message'])

# Creating sentiment labels
# Ratings 1 and 2 -> Negative (0)
# Rating 3 -> Neutral (undefined for now)
# Rating 4 and 5 (1)
def score_to_sentiment(score):
    if score in [1, 2]:
        return 0 # Negative
    elif score in [4, 5]:
        return 1 # Positive
    else:
        return None # Neutral/undefined for now
    
df_reviews['sentiment'] = df_reviews['review_score'].apply(score_to_sentiment)

# Removing the neutral reviews
df_reviews = df_reviews.dropna(subset=['sentiment'])

# Selecting only the reviews that interest
df_final = df_reviews[['review_comment_message', 'sentiment']].copy()
df_final.rename(columns={'review_comment_message': 'text', 'sentiment':'label'}, inplace=True)

# Converting the 'label' column type to integer
df_final['label'] = df_final['label'].astype(int)

# The dataset is too big
# We will take a more balanced dataset, for faster training
sample_size = 5000 # 5000 positives and 5000 negatives
df_sampled = df_final.groupby('label').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)

print(df_sampled['label'].value_counts())
print(df_sampled.head())

label
0    5000
1    5000
Name: count, dtype: int64
                                                text  label
0                                           n chegou      0
1  Um dos relogios veio aberto sem tampa e sem fu...      0
2  O produto chegou em 2 entregas, só que bem mal...      0
3  Ao invés de vocês se preocuparem em resolver o...      0
4  Estou aguardando entrega meu produto desde o d...      0


C:\Users\leors\AppData\Local\Temp\ipykernel_18120\3624836757.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['sentiment'] = df_reviews['review_score'].apply(score_to_sentiment)
C:\Users\leors\AppData\Local\Temp\ipykernel_18120\3624836757.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_final.groupby('label').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)


## 3. Preparing the Data for BERT

In [5]:
from datasets import Dataset
from transformers import AutoTokenizer

# Loading the dataser from the Pandas DataFrame
dataset = Dataset.from_pandas(df_sampled)

# Loading the BERT tokenizer in Portuguese
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to tokenize the texts
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Applying the tokenization to the whole dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Splitting into training and test (80% training, 20% test)
final_datasets = tokenized_datasets.train_test_split(test_size=0.2)

print(final_datasets)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 10000/10000 [00:00<00:00, 22335.80 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})


## 4. Fine-tuning the BERT Model